In [2]:
import os
os.environ['CASTLE_BACKEND'] = 'pytorch'

In [3]:
from castle.common import GraphDAG
from castle.metrics import MetricsDAG
from castle.datasets import DAG, IIDSimulation
from castle.algorithms import Notears, NotearsNonlinear
import numpy as np

2023-11-16 18:51:20,829 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.


learning causality of self generated data

In [10]:
np.random.seed(2)
x = np.random.normal(0,1, 800)

np.random.seed(24)
l = np.random.normal(0,1, 800) 
y = [0.5*a + b for a, b in zip(x, l)]

In [13]:
X = list(zip(x,y))
X = np.array(X)

(800, 2)

In [14]:
nt = Notears()
nt.learn(X)

2023-11-16 19:06:11,439 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:195] - INFO: [start]: n=800, d=2, iter_=100, h_=1e-08, rho_=1e+16
2023-11-16 19:06:11,455 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:206] - INFO: [iter 0] h=6.591e-03, loss=1.120, rho=1.0e+00
2023-11-16 19:06:11,460 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:206] - INFO: [iter 1] h=6.574e-03, loss=0.991, rho=1.0e+00
2023-11-16 19:06:11,465 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:206] - INFO: [iter 1] h=6.422e-03, loss=0.992, rho=1.0e+01
2023-11-16 19:06:11,472 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:206] - INFO: [iter 1] h=5.412e-03, loss=0.994, rho=1.0e+02
2023-11-16 19:06:11,481 - c:\Users\yur

In [15]:
nt.causal_matrix

Tensor([[0, 1],
        [0, 0]])

learning causality of package generated data

In [151]:
weighted_random_dag = DAG.erdos_renyi(n_nodes=2, n_edges=1, weight_range=(0.5, 2.0), seed = 1)
dataset = IIDSimulation(W=weighted_random_dag, n=2000, method='linear', sem_type='gauss')
true_dag, X = dataset.B, dataset.X

2023-11-13 19:33:29,517 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\datasets\simulator.py[line:270] - INFO: Finished synthetic dataset


In [152]:
nt = Notears()
nt.learn(X)

2023-11-13 19:33:30,201 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:195] - INFO: [start]: n=2000, d=2, iter_=100, h_=1e-08, rho_=1e+16
2023-11-13 19:33:30,215 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:206] - INFO: [iter 0] h=6.395e-02, loss=1.416, rho=1.0e+00
2023-11-13 19:33:30,228 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:206] - INFO: [iter 1] h=6.011e-02, loss=0.898, rho=1.0e+00
2023-11-13 19:33:30,242 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:206] - INFO: [iter 1] h=4.383e-02, loss=0.916, rho=1.0e+01
2023-11-13 19:33:30,257 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:206] - INFO: [iter 1] h=2.017e-02, loss=1.100, rho=1.0e+02
2023-11-13 19:33:30,285 - c:\Users\yu

In [153]:
nt.causal_matrix

Tensor([[0, 0],
        [1, 0]])

In [154]:
true_dag

array([[0, 0],
       [1, 0]])

In [138]:
met = MetricsDAG(nt.causal_matrix, true_dag)
print(met.metrics)

{'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 1, 'precision': 1.0, 'recall': 1.0, 'F1': 1.0, 'gscore': 1.0}


c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\metrics\evaluation.py:193: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  num_tp =  (W_p + W_true).applymap(lambda elem:1 if elem==2 else 0).sum(axis=1).sum()
c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\metrics\evaluation.py:195: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  num_fn_r = (W_p - W_true).applymap(lambda elem:1 if elem==1 else 0).sum(axis=1).sum()
c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\metrics\evaluation.py:221: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  TP = (W_p + W_true).applymap(lambda elem:1 if elem==2 else 0).sum(axis=1).sum()


load data

In [6]:

file_path = './Data/pair0001.txt'

X = np.loadtxt(file_path)


In [7]:
X

array([[ 2.050e+02,  9.700e+00],
       [ 4.600e+01,  8.200e+00],
       [ 7.940e+02,  6.400e+00],
       [ 3.250e+02,  8.100e+00],
       [ 5.000e+02,  6.200e+00],
       [ 2.150e+02,  9.400e+00],
       [ 3.830e+02,  7.800e+00],
       [ 5.400e+01,  8.300e+00],
       [ 6.300e+02,  6.600e+00],
       [ 4.130e+02,  8.300e+00],
       [ 4.810e+02,  7.500e+00],
       [ 4.200e+01,  7.900e+00],
       [ 2.420e+02,  8.400e+00],
       [ 1.640e+02,  8.500e+00],
       [ 4.620e+02,  8.000e+00],
       [ 5.600e+02,  7.600e+00],
       [ 4.000e+00,  8.400e+00],
       [ 6.220e+02,  7.200e+00],
       [ 2.390e+02,  8.500e+00],
       [ 4.000e+01,  8.600e+00],
       [ 3.300e+02,  7.700e+00],
       [ 7.890e+02,  6.600e+00],
       [ 4.500e+02,  8.000e+00],
       [ 4.000e+01,  8.800e+00],
       [ 2.520e+02,  9.800e+00],
       [ 4.450e+02,  7.000e+00],
       [ 6.000e+01,  8.800e+00],
       [ 5.800e+01,  8.900e+00],
       [ 3.600e+01,  9.300e+00],
       [ 4.900e+01,  9.400e+00],
       [ 8

In [156]:
X.shape

(721, 2)

learning causality of real data (contrary)

linear SEM

In [157]:
nt = Notears(w_threshold=0.2)
nt.learn(X)

2023-11-13 19:37:31,713 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:195] - INFO: [start]: n=721, d=2, iter_=100, h_=1e-08, rho_=1e+16
2023-11-13 19:37:31,724 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:206] - INFO: [iter 0] h=6.455e-02, loss=80409.127, rho=1.0e+00
2023-11-13 19:37:31,729 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:206] - INFO: [iter 1] h=6.451e-02, loss=51511.375, rho=1.0e+00
2023-11-13 19:37:31,736 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:206] - INFO: [iter 1] h=6.412e-02, loss=51511.394, rho=1.0e+01
2023-11-13 19:37:31,743 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\linear.py[line:206] - INFO: [iter 1] h=6.124e-02, loss=51511.582, rho=1.0e+02
2023-11-13 19:37:31,75

In [158]:
nt.causal_matrix

Tensor([[0, 1],
        [0, 0]])

non-linear SEM

In [159]:
nt = NotearsNonlinear()
nt.learn(X)

2023-11-13 19:38:16,077 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\torch\nonlinear.py[line:137] - INFO: GPU is unavailable.
2023-11-13 19:38:16,079 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\torch\nonlinear.py[line:237] - INFO: [start]: n=721, d=2, iter_=100, h_=1e-08, rho_=1e+16
2023-11-13 19:38:28,391 - c:\Users\yurou\anaconda3\envs\NOTEARS\Lib\site-packages\castle\algorithms\gradient\notears\torch\nonlinear.py[line:249] - INFO: FINISHED


In [161]:
t = nt.causal_matrix
t

Tensor([[0, 1],
        [0, 0]])